In this problem, you will be using the Baseball Databank provided by Sean Lahman. This data is already in the data/baseballdatabank2017.1/core/ directory. It contains a collection of tables. The immediate goal will be to create a dataframe that has the following information for each baseball player:

Batting statistics (to be described)
Fielding statistics (to be described)
Pitching statistics (to be described)
Their salaries
The teams they played for
Their full names: First Middle Last
Their heights and weights
The dataset has a data dictionary available at:

../data/baseballdatabank-2017.1/core/readme2014.txt
The data and tables you will need are:

Batting.csv
The number of games played and at bats
The number of runs, hits, doubles, triples, homeruns, RBIs, strikeouts, and times hit by

Fielding.csv
The number of games played
The number of opponents put out, assisted outs, and fielding errors

Master.csv
Their full names
Their heights and weights

Pitching.csv
The number of games played, won, lost
The number of strikeouts, hits, earned runs, homeruns, and batters hit by pitches

Salaries.csv
The players salary

Teams.csv
The name of the player's team
The year the team was named its name


In [ ]:
import pandas as pd

def load_columns_of_interest(path_to_databank_core = 'data/baseballdatabank2017.1/core/'):
    (appearances_file, batting_file, 
     fielding_file, master_file, pitching_file, 
     salaries_file, teams_file) = [path_to_databank_core + table_file
                                   for table_file in ["Appearances.csv", "Batting.csv", 
                                                      "Fielding.csv", "Master.csv", 
                                                      "Pitching.csv", "Salaries.csv", 
                                                      "Teams.csv"]] 
    batting = pd.read_csv(batting_file,header=0)[['playerID','yearID','teamID','G','AB','R','H','2B','3B','HR','RBI','SO','HBP']]
    master = pd.read_csv(master_file,header=0)[['playerID','nameFirst','nameLast','weight','height']]
    pitching = pd.read_csv(pitching_file,header=0)[['playerID','yearID','teamID','W','L','G','H','ER','HR','HBP','SO']]
    fielding = pd.read_csv(fielding_file,header=0)[['playerID','yearID','teamID','G','PO','A','E']]
    salaries = pd.read_csv(salaries_file,header=0)[['yearID','teamID','playerID','salary']]
    teams = pd.read_csv(teams_file,header=0)[['teamID','name','yearID']]
    appearances = pd.read_csv(appearances_file, header = 0)[['yearID','teamID','playerID']]
    
    return [batting, master, pitching, fielding, salaries, teams, appearances]

In [38]:
baseball_data = load_columns_of_interest('data/baseballdatabank2017.1/core/')

#Isolate list items to create dataframes

batting = baseball_data[0]
master =baseball_data[1]
pitching = baseball_data[2]
fielding = baseball_data[3]
salaries = baseball_data[4]
teams = baseball_data[5]
appearances = baseball_data[6]

#Change from list item to DataFrame 

pitch = pd.DataFrame(pitching)
bat = pd.DataFrame(batting)
mast_df = pd.DataFrame(master)
field = pd.DataFrame(fielding)
appear = pd.DataFrame(appearances)
team = pd.DataFrame(teams)
salary = pd.DataFrame(salaries)

#Group for next step

all = (pitch, bat, mast_df, field, appear, team, salary)


In [39]:
#Filter by year
def FilterByYear(year, all):
  
    ff = field[field['yearID']==year]
    bf = bat[bat['yearID']==year]
    pf = pitch[pitch['yearID']==year]
    af = appear[appear['yearID']==year]
    mf = mast_df
    tf = team[team['yearID']==year]
    sf = salary[salary['yearID']==year]
    
    
    return(ff, bf, pf, af, mf, tf, sf)

In [43]:
#Sanity Check

print(FilterByYear(1987, all)[-1].head())

      yearID teamID   playerID  salary
1288    1987    ATL  ackerji01  350000
1289    1987    ATL  alexado01  650000
1290    1987    ATL  assenpa01   80000
1291    1987    ATL  barkele01  890000
1292    1987    ATL  benedbr01  615000


In [87]:
def MergeTables(year, all):
    
    (ff, bf, pf, af, mf, tf, sf) = FilterByYear(year, all)
    
 
    merged = af.merge(bf, on=['playerID','yearID','teamID'], how='left')
    #---your code stops here---
    
    
    ## merge on the fielding data
    #---your code starts here---
    merged = merged.merge(ff, on=['playerID','yearID','teamID'], how='left', suffixes=[None, '_p'])
    #---your code stops here---
    
    
    ## merge on the player data
    #---your code starts here---
    merged = merged.merge(mf, on=['playerID'], how='left')

    #---your code stops here---
    
    ## merge on the pitching data
    #---your code starts here---
    merged = merged.merge(pf, on=['playerID','yearID','teamID'], how='left', suffixes=[None, '_p'])

    #---your code stops here---
    
    ## merge on the salaries data
    #---your code starts here---
    merged = merged.merge(sf, on=['playerID','yearID','teamID'], how='left')
 
    #---your code stops here---
    
    ## merge on the teams data
    #---your code starts here---
    merged = merged.merge(tf, on=['yearID','teamID'], how='left')
 
    #---your code stops here---

    return merged
    
    

In [88]:
#Sanity check

merged = MergeTables(2008, all)
print(merged.head())

   yearID teamID   playerID    G   AB    R    H  2B  3B  HR  ...    W    L  \
0    2008    BOS  aardsda01   47    1    0    0   0   0   0  ...  4.0  2.0   
1    2008    HOU  abercre01   34   55   10   17   5   0   2  ...  NaN  NaN   
2    2008    NYA  abreubo01  156  609  100  180  39   4  20  ...  NaN  NaN   
3    2008    TOR  accarje01   16    0    0    0   0   0   0  ...  0.0  3.0   
4    2008    NYA  aceveal01    6    0    0    0   0   0   0  ...  1.0  0.0   

    G_p   H_p    ER  HR_p  HBP_p  SO_p      salary               name  
0  47.0  49.0  30.0   4.0    5.0  49.0    403250.0     Boston Red Sox  
1   NaN   NaN   NaN   NaN    NaN   NaN         NaN     Houston Astros  
2   NaN   NaN   NaN   NaN    NaN   NaN  16000000.0   New York Yankees  
3  16.0  15.0   9.0   1.0    1.0   5.0    392200.0  Toronto Blue Jays  
4   6.0  25.0   8.0   4.0    0.0  16.0         NaN   New York Yankees  

[5 rows x 31 columns]


In [89]:
#Counting NaN values in the salary column
merged["salary"].isna().sum()

729

In [90]:
#Replace NaN values with zeros 
merged.fillna(value=0.0,inplace=True)

In [91]:
merged['salary'].isna().sum()

0